In [12]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import sklearn
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# from sklearn.feature_selection import RFE
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

# 读取数据

In [3]:
def unique_value(df: pd.DataFrame) -> np.array:
    df_specific = df.drop(list(df.columns[:2]), axis=1, inplace=False)
    temp = df_specific.to_numpy()
    return np.unique(temp)

In [4]:
train_df = pd.read_csv("../dataset/recipes_train.csv")
test_df = pd.read_csv("../dataset/recipes_test.csv")

In [5]:
train_df.shape, test_df.shape

((1469, 385), (979, 384))

In [6]:
train_df.head()

,id,cuisine,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,1,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,chinese,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,chinese,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,korean,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,thai,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
test_df.head()

,id,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,artemisia,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 单model

In [8]:
food_index = {}
food_count = {} # key为类, value为对应类的出现次数
loc_index = {} # key为类, value为其所有数据的索引
for _, group in train_df.groupby(by='cuisine'):
    # name, group = specific data -> DataFrame
    name = group['cuisine'].head(1).item()
    group.drop(["id", 'cuisine'], axis=1, inplace=True)
    loc_index[name] = group.index
    food_index[name] = group.sum().to_dict()
    food_count[name] = len(group)
food_index.keys() # key为类, value为类中各元素总和 dict: (column:sum_value)

dict_keys(['chinese', 'indian', 'japanese', 'korean', 'thai'])

In [9]:
from sklearn.decomposition import PCA
features_names = train_df.columns[2:]
labels = train_df.loc[:, 'cuisine']
pca = PCA(n_components=50)
reduced_features = pca.fit_transform(train_df.loc[:, features_names])

## 训练

In [10]:
# 将用于训练的内容提取出来
# train_x = train_df[result_indices].values
train_x = reduced_features
train_y = train_df["cuisine"].values

In [12]:
# 分割训练集验证集, 42更好。。
X_train, X_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [39]:
params_sklearn = {
    'learning_rate':0.01,
    # 'num_leaves':2^11-1,    
    # 'max_depth':11,
    
    'reg_alpha':0.1,
    'reg_lambda':0.1,   

    'subsample':0.9,
    'colsample_bytree':0.9,
    # 'class_weight':weight
}
clf = lgb.LGBMClassifier(**params_sklearn)
clf.fit(X_train, y_train, early_stopping_rounds=50, eval_set=[(X_valid, y_valid)], verbose=10)

d:\Anaconda3\envs\pytorch\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
d:\Anaconda3\envs\pytorch\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[10]	valid_0's multi_logloss: 1.39233
[20]	valid_0's multi_logloss: 1.28181
[30]	valid_0's multi_logloss: 1.19178
[40]	valid_0's multi_logloss: 1.11516
[50]	valid_0's multi_logloss: 1.0524
[60]	valid_0's multi_logloss: 0.998395
[70]	valid_0's multi_logloss: 0.951137
[80]	valid_0's multi_logloss: 0.911298
[90]	valid_0's multi_logloss: 0.875386
[100]	valid_0's multi_logloss: 0.843567


LGBMClassifier(colsample_bytree=0.9, learning_rate=0.01, reg_alpha=0.1,
               reg_lambda=0.1, subsample=0.9)

In [46]:
grid = GridSearchCV(lgb.LGBMClassifier(n_estimators=500, objective='multiclass', learning_rate=0.01), \
    param_grid={
    'max_depth':[3, 5, 6, 7, 9, 12, 15, 17, 25],
    'subsample':[0.8, 0.9, 1.0],
    'colsample_bytree':[0.8, 0.9, 1.0],
    'reg_alpha':[0, 0.1, 0.5, 1],
}, cv=4)
grid.fit(train_x, train_y)

GridSearchCV(cv=4,
             estimator=LGBMClassifier(learning_rate=0.01, n_estimators=500,
                                      objective='multiclass'),
             param_grid={'colsample_bytree': [0.8, 0.9, 1.0],
                         'max_depth': [3, 5, 6, 7, 9, 12, 15, 17, 25],
                         'reg_alpha': [0, 0.1, 0.5, 1],
                         'subsample': [0.8, 0.9, 1.0]})

In [48]:
clf = grid.best_estimator_
clf.fit(X_train, y_train)
print("train accuracy:", accuracy_score(clf.predict(X_train), y_train))
print("valid accuracy:", accuracy_score(clf.predict(X_valid), y_valid))

LGBMClassifier(colsample_bytree=0.8, learning_rate=0.01, max_depth=9,
               n_estimators=500, objective='multiclass', reg_alpha=0,
               subsample=0.8)

In [13]:
best_result = 0
for n_component in [50, 100, 150, 200, 250, 300, 350]:
    features_names = train_df.columns[2:]
    labels = train_df.loc[:, 'cuisine']
    pca = PCA(n_components=n_component)
    reduced_features = pca.fit_transform(train_df.loc[:, features_names])
    train_x = reduced_features
    train_y = train_df["cuisine"].values
    grid = GridSearchCV(lgb.LGBMClassifier(n_estimators=500, objective='multiclass'), \
    param_grid={
        'learning_rate':[0.01, 0.015, 0.025, 0.05, 0.1],
        'max_depth':[3, 5, 6, 7, 9, 12, 15, 17, 25],
        'subsample':[0.8, 0.9, 1.0],
        'colsample_bytree':[0.8, 0.9, 1.0],
        'reg_alpha':[0, 0.1, 0.5, 1],
    }, cv=4)
    grid.fit(train_x, train_y)
    if grid.best_score_ > best_result:
        best_result = grid.best_score_
        best_estimator = grid.best_estimator_
        best_n_component = n_component

In [ ]:
# 分割训练集验证集, 42更好。。
X_train, X_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.2, random_state=42)
best_estimator.fit(X_train, y_train)
print("train accuracy:", accuracy_score(clf.predict(X_train), y_train))
print("valid accuracy:", accuracy_score(clf.predict(X_valid), y_valid))